In [1]:
import pickle
import scipy as sp
import igl
import robust_laplacian

import os.path as osp
import argparse
import numpy as np
import torch
import smplx
import glob
import cv2
import os
import pyrender
import trimesh
import json
from tqdm import tqdm

In [2]:
with open('../00122/Outer/Take9/SMPLX/mesh-f00011_smplx.pkl', 'rb') as file:
    smplx_data = pickle.load(file)
smplx_data

{'global_orient': array([ 0.00492813,  1.6237236 , -0.0699005 ], dtype=float32),
 'body_pose': array([-4.19760346e-02,  9.74555463e-02,  1.90463886e-01, -7.47566670e-02,
        -7.21446872e-02, -1.57268777e-01, -4.56898585e-02,  4.43277182e-03,
         5.37694916e-02, -6.97312132e-02,  8.66318271e-02,  9.14673135e-02,
        -1.26157841e-02, -6.44140169e-02, -1.91337466e-02,  5.14160208e-02,
        -1.75482202e-02,  8.12641811e-03,  1.33622453e-01,  5.47491647e-02,
        -1.69454706e-05,  1.37154654e-01, -3.88579182e-02,  1.61871758e-05,
         8.12471882e-02, -8.73514544e-03,  2.21590884e-02,  1.58502371e-05,
         1.57070062e-05,  1.54597165e-05,  1.54723148e-05,  1.39889626e-05,
        -1.51485538e-05, -1.13553563e-02, -4.63176444e-02, -6.14181906e-02,
        -5.00713438e-02,  2.19184086e-01, -1.77930251e-01, -2.58259997e-02,
        -1.98528439e-01,  1.84636593e-01,  7.81371593e-02, -1.03415944e-01,
         8.63157436e-02, -8.68193135e-02,  5.43496907e-02, -8.56566191

In [3]:
body_model_param = {
    'betas': np.expand_dims(smplx_data['betas'],0),
    'expression': np.expand_dims(smplx_data['expression'],0),
    'transl': np.expand_dims(smplx_data['transl'],0),
    'global_orient': np.expand_dims(smplx_data['global_orient'], 0),
    'body_pose': np.expand_dims(smplx_data['body_pose'], 0),
    'jaw_pose': np.expand_dims(smplx_data['jaw_pose'], 0),
    'leye_pose': np.expand_dims(smplx_data['leye_pose'], 0),
    'reye_pose': np.expand_dims(smplx_data['reye_pose'], 0),
    'left_hand_pose': np.expand_dims(smplx_data['left_hand_pose'], 0),
    'right_hand_pose': np.expand_dims(smplx_data['right_hand_pose'], 0)
}

In [4]:
for key, value in body_model_param.items():
    print(f'{key} shape: {value.shape}')

betas shape: (1, 10)
expression shape: (1, 10)
transl shape: (1, 3)
global_orient shape: (1, 3)
body_pose shape: (1, 63)
jaw_pose shape: (1, 3)
leye_pose shape: (1, 3)
reye_pose shape: (1, 3)
left_hand_pose shape: (1, 12)
right_hand_pose shape: (1, 12)


In [5]:
body_model_param_tensor = {
    key: torch.tensor(body_model_param[key]) for key in body_model_param.keys()
}

In [6]:
model_folder = '/home/yeyiqi/Documents/repos/GaussianAvatar/assets/smpl_files'
model_type = 'smplx'

kwargs = dict(
        gender='neutral',
        num_betas=10,
        use_face_contour=True,
        num_pca_comps=12,
        use_pca=True,
        batch_size=1,
        ext = 'pkl'
)
model = smplx.create(
    model_path = model_folder,
    model_type = model_type,
    **kwargs
)

/home/yeyiqi/.conda/envs/MyFirst/lib/python3.10/site-packages/smplx/body_models.py:1009: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  dynamic_lmk_bary_coords = torch.tensor(


In [7]:
output = model(**body_model_param_tensor)

torch.Size([1, 55, 3, 3])


In [8]:
vertices = output.vertices.detach().cpu().numpy().squeeze()
joints = output.joints.detach().cpu().numpy().squeeze()
faces = model.faces

In [9]:
vertices

array([[-0.21869732,  1.9355723 ,  0.0109391 ],
       [-0.217415  ,  1.9345828 ,  0.00696669],
       [-0.21708615,  1.9359715 ,  0.006282  ],
       ...,
       [-0.16592452,  1.9516644 ,  0.11792305],
       [-0.166757  ,  1.9523582 ,  0.11572537],
       [-0.16712812,  1.9530214 ,  0.11338915]], dtype=float32)

In [10]:
faces

array([[    3,     1,     0],
       [    7,     5,     4],
       [   12,    14,    13],
       ...,
       [ 9944, 10097, 10084],
       [ 9940, 10084, 10071],
       [10071, 10058,  9932]], dtype=uint32)

In [11]:
with open('smplx.ply', 'w') as file:
    file.write(f'ply\n')
    file.write(f"format ascii 1.0\n")
    file.write(f"element vertex {len(vertices)}\n")
    file.write(f"property float x\n")
    file.write(f"property float y\n")
    file.write(f"property float z\n")
    file.write(f"element face {len(faces)}\n")
    file.write(f"property list uchar int vertex_indices\n")
    file.write(f"end_header\n")
    for i in range(len(vertices)):
        vertex = vertices[i]
        file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")
    for face in faces:
        file.write(f"3 {face[0]} {face[1]} {face[2]}\n")

In [12]:
with open('/home/yeyiqi/Documents/repos/GaussianAvatar/assets/smpl_files/smplx/SMPLX_NEUTRAL.pkl', 'rb') as file:
    model_data = pickle.load(file, encoding='latin1')
model_data

{'dynamic_lmk_bary_coords': [array([[0.22900535, 0.58840868, 0.18258597],
         [0.52562196, 0.44520835, 0.0291697 ],
         [0.62050531, 0.16457369, 0.214921  ],
         [0.02810657, 0.6297006 , 0.34219283],
         [0.2320207 , 0.24835697, 0.51962233],
         [0.341529  , 0.43981666, 0.21865434],
         [0.32329773, 0.35568534, 0.32101693],
         [0.39990793, 0.43517089, 0.16492117],
         [0.62985068, 0.35881678, 0.01133253],
         [0.391958  , 0.16314388, 0.44489813],
         [0.31645411, 0.37635828, 0.30718761],
         [0.26717159, 0.43234462, 0.30048379],
         [0.29556018, 0.16840894, 0.53603088],
         [0.10611214, 0.60458513, 0.28930273],
         [0.29643883, 0.68466573, 0.01889544],
         [0.16667023, 0.47798314, 0.35534663],
         [0.12019503, 0.20126353, 0.67854144]]),
  array([[0.30805641, 0.5211352 , 0.17080839],
         [0.43377128, 0.28416408, 0.28206464],
         [0.55549374, 0.03407959, 0.41042666],
         [0.17123556, 0.5496474

In [13]:
model_data.keys()

dict_keys(['dynamic_lmk_bary_coords', 'hands_componentsl', 'ft', 'lmk_faces_idx', 'f', 'J_regressor', 'hands_componentsr', 'kintree_table', 'hands_coeffsr', 'joint2num', 'hands_meanl', 'lmk_bary_coords', 'weights', 'posedirs', 'dynamic_lmk_faces_idx', 'part2num', 'vt', 'hands_meanr', 'hands_coeffsl', 'v_template', 'shapedirs'])

In [41]:
model_data['hands_componentsl'].shape

(45, 45)

In [15]:
v_template = torch.from_numpy(model_data['v_template'])
shapedirs = torch.from_numpy(model_data['shapedirs'])
posedirs = torch.from_numpy(model_data['posedirs'])
J_regressor = torch.from_numpy(model_data['J_regressor'])
parents = torch.from_numpy(model_data['kintree_table'])
weights = torch.from_numpy(model_data['weights'])
parents = parents[0]
parents[0] = -1
parents

tensor([-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14,
        16, 17, 18, 19, 15, 15, 15, 20, 25, 26, 20, 28, 29, 20, 31, 32, 20, 34,
        35, 20, 37, 38, 21, 40, 41, 21, 43, 44, 21, 46, 47, 21, 49, 50, 21, 52,
        53])

In [16]:
pose1_data = smplx_data

In [17]:
for key, value in pose1_data.items():
    tensor_value = torch.from_numpy(value)
    pose1_data[key] = tensor_value

In [18]:
pose1_data['left_hand_pose'] = torch.einsum(
    'bi, ij->bj', [pose1_data['left_hand_pose'].reshape(1,-1), torch.from_numpy(model_data['hands_componentsl'][:12])]
)
pose1_data['right_hand_pose'] = torch.einsum(
    'bi, ij->bj', [pose1_data['right_hand_pose'].reshape(1,-1), torch.from_numpy(model_data['hands_componentsr'][:12])]
)

In [19]:
print(pose1_data['right_hand_pose'].shape)
print(pose1_data['left_hand_pose'].shape)

torch.Size([1, 45])
torch.Size([1, 45])


In [20]:
full_pose1 = torch.cat([
    pose1_data['global_orient'].reshape(-1,1,3),
    pose1_data['body_pose'].reshape(-1, 21, 3),
    pose1_data['jaw_pose'].reshape(-1, 1, 3),
    pose1_data['leye_pose'].reshape(-1, 1, 3),
    pose1_data['reye_pose'].reshape(-1, 1, 3),
    pose1_data['left_hand_pose'].reshape(-1, 15, 3),
    pose1_data['right_hand_pose'].reshape(-1, 15, 3)
], dim=1).reshape(-1, 165)
full_pose1.shape

torch.Size([1, 165])

In [21]:
shape_components1 = torch.cat([pose1_data['betas'].reshape(-1, 10), pose1_data['expression'].reshape(-1, 10)], dim=-1)
shape_components1.shape

torch.Size([1, 20])

In [22]:
shapedirs1 = torch.cat([
    shapedirs[:,:,:10],
    shapedirs[:,:,300:310]
], dim=-1)
shapedirs1.shape

torch.Size([10475, 3, 20])

In [23]:
shape_components1 = shape_components1.double()
full_pose1 = full_pose1.double()
v_template = v_template.double()
posedirs = posedirs.double()
J_regressor = J_regressor.double()
weights = weights.double()

In [24]:
num_pose_basia = posedirs.shape[-1]
posedirs = torch.reshape(posedirs, [-1, num_pose_basia]).T
posedirs.shape

torch.Size([486, 31425])

In [25]:
pose1_vertices, pose1_joints = smplx.lbs.lbs(shape_components1, full_pose1, v_template, shapedirs1, posedirs, J_regressor, parents, weights)

torch.Size([1, 55, 3, 3])


In [26]:
pose1_data['transl'].shape

torch.Size([3])

In [27]:
pose1_vertices

tensor([[[ 0.0517,  0.2806, -0.0494],
         [ 0.0530,  0.2796, -0.0533],
         [ 0.0533,  0.2810, -0.0540],
         ...,
         [ 0.1045,  0.2967,  0.0576],
         [ 0.1036,  0.2974,  0.0554],
         [ 0.1033,  0.2980,  0.0531]]], dtype=torch.float64)

In [28]:
pose1_data['transl']

tensor([-0.2704,  1.6550,  0.0603])

In [29]:
pose1_transled_vertices = pose1_vertices + pose1_data['transl']

In [30]:
from smplx import vertex_ids as VERTEX_IDS
vertex_joint_selector = smplx.vertex_joint_selector.VertexJointSelector(VERTEX_IDS.vertex_ids['smplx'])

In [31]:
pose1_joints = vertex_joint_selector(pose1_vertices, pose1_joints)
pose1_joints.shape

torch.Size([1, 76, 3])

In [32]:
with open('smplx_pose1_transl.ply', 'w') as file:
    file.write(f'ply\n')
    file.write(f"format ascii 1.0\n")
    file.write(f"element vertex {len(pose1_transled_vertices[0])}\n")
    file.write(f"property float x\n")
    file.write(f"property float y\n")
    file.write(f"property float z\n")
    file.write(f"element face {len(faces)}\n")
    file.write(f"property list uchar int vertex_indices\n")
    file.write(f"end_header\n")
    for i in range(len(pose1_transled_vertices[0])):
        vertex = pose1_transled_vertices[0][i]
        file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")
    for face in faces:
        file.write(f"3 {face[0]} {face[1]} {face[2]}\n")

In [33]:
output.right_hand_pose.reshape(-1,15,3)

tensor([[[-2.1854e-01, -1.6936e-01, -2.3003e-01],
         [-2.6229e-02, -1.0350e-01, -2.8875e-01],
         [ 2.0749e-01, -2.0073e-02, -1.0193e-02],
         [ 4.6857e-02, -3.3182e-02,  3.3681e-01],
         [ 1.3024e-01,  1.2062e-02, -9.4608e-01],
         [-2.2884e-02, -8.6022e-02, -1.8402e-01],
         [ 3.3502e-01,  1.5485e-01,  1.3337e-01],
         [ 1.0602e-01, -1.0020e-01, -4.2191e-01],
         [ 1.3114e-01, -1.8916e-04, -1.4390e-01],
         [ 2.4316e-01,  1.1869e-01,  2.6694e-01],
         [ 3.7406e-02,  3.6220e-02, -9.5565e-01],
         [ 1.9376e-02, -1.7300e-01, -1.9255e-01],
         [-8.1973e-01,  3.4616e-01, -5.2833e-01],
         [ 9.3048e-02, -8.9074e-02,  6.8225e-01],
         [-1.6209e-01, -1.5129e-01, -6.2044e-01]]])

In [34]:
pose1_data['right_hand_pose'].reshape(-1,15,3)

tensor([[[-2.1854e-01, -1.6936e-01, -2.3003e-01],
         [-2.6229e-02, -1.0350e-01, -2.8875e-01],
         [ 2.0749e-01, -2.0073e-02, -1.0193e-02],
         [ 4.6857e-02, -3.3182e-02,  3.3681e-01],
         [ 1.3024e-01,  1.2062e-02, -9.4608e-01],
         [-2.2884e-02, -8.6022e-02, -1.8402e-01],
         [ 3.3502e-01,  1.5485e-01,  1.3337e-01],
         [ 1.0602e-01, -1.0020e-01, -4.2191e-01],
         [ 1.3114e-01, -1.8916e-04, -1.4390e-01],
         [ 2.4316e-01,  1.1869e-01,  2.6694e-01],
         [ 3.7406e-02,  3.6220e-02, -9.5565e-01],
         [ 1.9376e-02, -1.7300e-01, -1.9255e-01],
         [-8.1973e-01,  3.4616e-01, -5.2833e-01],
         [ 9.3048e-02, -8.9074e-02,  6.8225e-01],
         [-1.6209e-01, -1.5129e-01, -6.2044e-01]]])

In [38]:
output

SMPLXOutput(vertices=tensor([[[-0.2187,  1.9356,  0.0109],
         [-0.2174,  1.9346,  0.0070],
         [-0.2171,  1.9360,  0.0063],
         ...,
         [-0.1659,  1.9517,  0.1179],
         [-0.1668,  1.9524,  0.1157],
         [-0.1671,  1.9530,  0.1134]]]), joints=tensor([[[-2.6766e-01,  1.2895e+00,  6.5571e-02],
         [-3.0173e-01,  1.1958e+00,  1.7065e-02],
         [-2.9103e-01,  1.1881e+00,  1.2998e-01],
         [-2.8662e-01,  1.4004e+00,  6.4701e-02],
         [-3.2605e-01,  8.3342e-01, -9.3222e-02],
         [-2.9539e-01,  8.4566e-01,  2.4486e-01],
         [-2.9121e-01,  1.5422e+00,  5.8276e-02],
         [-3.3478e-01,  4.4037e-01, -1.4596e-01],
         [-2.9572e-01,  4.5020e-01,  3.1577e-01],
         [-2.6403e-01,  1.5961e+00,  6.9024e-02],
         [-2.4092e-01,  3.9186e-01, -2.3703e-01],
         [-1.8793e-01,  3.9406e-01,  3.8092e-01],
         [-2.6468e-01,  1.7616e+00,  8.8017e-02],
         [-2.5671e-01,  1.6827e+00,  2.5949e-02],
         [-2.6008e-01,  1.6